In [ ]:
# Namespace
        
def graphviz_subtrees(inodes):
    print "digraph G {"
    for i in inodes:
        children = str(inodes[i]).replace(',', ' ')
        children = children.replace('[', '').replace(']', '')
        children = children.replace('\'', '\"').replace(':', '\\n')
        print "  \"" + i.replace(':', '\\n') + "\" -> {" + children + "};"
    print "}"
#sys.stdout = open('namespace.dot', 'w')
#graphviz_subtrees(inodes)

In [ ]:
def build_subtrees(ops):
    inodes = {} # all inodes, i.e. the namespace
    roots  = [] # potential roots, i.e. inodes with parents
    childs = [] # list of children
    for _, op in ops.iterrows():
        msg    = op['msg'].split(' ')
        child  = msg[0].split('=')[1][:-1][1:]#.split(':')[0]
        parent = msg[1].split('=')[1][:-1][1:]#.split(':')[0]
    
        if parent == child:
            continue
        if parent not in inodes:
            inodes[parent] = []
        inodes[parent].append(child)
    
        if parent not in roots:
            roots.append(parent)
        if child not in childs:
            childs.append(child)
    return inodes, roots, childs

# only dump roots that have no parents (i.e. not roots)
def index_subtrees(root, inodes, inomap, ino=1):
    inomap[root] = ino
    ino += 1
    for child in inodes[root]:
        if child in inodes:
            index_subtrees(child, inodes, inomap, ino=ino)
        else:
            inomap[child] = ino
            ino += 1
    return ino, inomap

def dump_subtrees(root, inodes, level=0, ino=1):
    if level == 0:
        print '  ' + root + ' ' + str(ino)
        ino += 1
    elif level == 1:
        print '  |_' + root + ' ' + str(ino)
        ino += 1
    else:
        print level*'  ' + '|_' +  root + ' ' + str(ino)
        ino += 1
    for child in inodes[root]:
        if child in inodes:
            dump_subtrees(child, inodes, level=level+1, ino=ino)
        else:
            print (level+1)*'  ' + '|_' + child + ' ' + str(ino)
            ino += 1
    return ino, inomap



ino = 1; inomap = {}
for df in [df0, df1, df2]:
    ops = df[(df['type'] == '02MKDIR') | (df['type'] == '01CREAT')]
    inodes, roots, childs = build_subtrees(ops)
    for r in roots:
        if r not in childs:
            ino, inomap = index_subtrees(r, inodes, inomap, ino=ino)
print len(inomap)

In [ ]:
def assign_namespace_keys(keys):
    accesses = []
    missed = 0
    for key in keys:
        try:
            keyid = inomap[key]
        except:
            keyid = 0
            missed += 1
        accesses.append(keyid)
    print "missed=" + str(missed)
    return accesses

def get_namespace_ops(df, op):
    ops = df[df['type'] == op]
    time = ops['time'].apply(lambda x: to_seconds(x)).as_matrix()
    keys = ops['msg'].apply(lambda x: x.split(' ')[0].split('=')[1][:-1][1:])
    if len(time) == 0:
        return [], []
    return time - time[0], assign_namespace_keys(keys)

fig, ax = plt.subplots(1, 1)#, figsize=(12, 24))
time, keys = get_namespace_ops(df1, '01CREAT')
ax.scatter(time, keys, s=2, label='01CREAT')
ax.set_xlabel("Time (sec)")
ax.set_title("Day ")
ax.legend(loc='upper left', fontsize=12)
ax.set_ylabel("File ID")

In [ ]:
fig, ax = plt.subplots(1, 1)
time, keys = get_namespace_ops(df2, '19ATIME')
ax.scatter(time, keys, s=2, label='19ATIME')
ax.set_xlabel("Time (sec)")
ax.set_title("Day ")
ax.legend(loc='upper left', fontsize=12)
ax.set_ylabel("File ID")
#ax.set_ylim(13000, 16000)